In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
#subjects or patient
subjects=pd.read_csv("/kaggle/input/subjects.csv")
#events 
events=pd.read_csv("/kaggle/input/events.csv")
#tasks 
task=pd.read_csv("/kaggle/input/tasks.csv")
#defog metadata has subject's data  in home 
defog_metadata=pd.read_csv("/kaggle/input/defog_metadata.csv")
#tdcsfog metadata has subject's data in lab 
tdcs_metadata=pd.read_csv("/kaggle/input/tdcsfog_metadata.csv")
#test data 
test = glob.glob('/kaggle/input/test/defog/**')
#train data
train_tdcs=glob.glob('/kaggle/input/train/tdcsfog/**')
train_defog=glob.glob('/kaggle/input/train/defog/**')

In [ ]:
#function to assign Id to a dataset row from its filename
def readfiles(file_path):
    df=pd.read_csv(file_path)
    df['Id'] = file_path.split("/")[-1].split(".")[0]
    
    return df
    

In [ ]:
#Test Data first
test_defog = pd.concat([readfiles(f) for f in test]).fillna(0);   
test_defog


In [ ]:
#giving id to train/defog data
train_defog = pd.concat([readfiles(f) for f in train_defog]).fillna(0);   
train_defog

In [ ]:
#subject,defog_metadata
subject_defog_metadata=pd.merge(subjects,defog_metadata,on="Subject",how="inner")
subject_defog_metadata

In [ ]:
#defog_metadata+subject+train_defog
defog_final=pd.merge(train_defog,subject_defog_metadata,on="Id",how="inner")
defog_final

In [ ]:
#function to reduce memory size
def reduce_memory_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024 ** 2 
    print("Memory usage became: ",mem_usg," MB")
    
    return df

In [ ]:
#reducing memory size
defog_final=reduce_memory_usage(defog_final)

In [ ]:
defog_final1=pd.merge(defog_final[:10000],events,on="Id",how="inner")
defog_final1

# defog_final1.groupby("Visit_y").count()


In [ ]:
defog_final1.info()

In [ ]:
columns_to_remove = ['Walking', 'Turn', 'StartHesitation','Valid','Task','Subject','Visit_y','Id']

defog_final1 = defog_final1.drop(columns=columns_to_remove)


In [ ]:
defog_final1.groupby("Type").count()


In [ ]:
#binarization of gender
defog_final1["Sex"]=defog_final1["Sex"].map({"M":1,"F":0})
defog_final1["Medication"]=defog_final1["Medication"].map({"on":1,"off":0})

# defog_final1["Type"]=defog_final1["Type"].map({"Walking":10,"Turn":11,"StartHesitation":12})


In [ ]:
# #Changing type into binary classed i.e 0 and 1
# defog_final1=pd.get_dummies(defog_final1, columns=['Type'])

In [ ]:
moved_column = defog_final1.pop("Type")
defog_final1["Type"] = moved_column
defog_final1

In [ ]:
#reduce defog_final1 
defog_final1=reduce_memory_usage(defog_final1)

In [ ]:
defog_final1.groupby("Type").count()

In [ ]:
X = defog_final1.iloc[:, :-1]  # Features
y = defog_final1.iloc[:, -1]   # Target variable


In [ ]:
y

**GAUSSIAN NAIVE BAYESIAN  WITH ROC,AuC****

**LATER**

In [ ]:
import numpy as np
from sklearn.preprocessing import label_binarize, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

class DecisionTreeID3:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        # Stopping criteria
        if self.max_depth is not None and depth == self.max_depth or n_classes == 1:
            return {'value': np.bincount(y).argmax()}
        # Find best split
        best_split = self._find_best_split(X, y)
        if best_split is None:
            return {'value': np.bincount(y).argmax()}
        # Split data
        left_indices = X[:, best_split['feature']] <= best_split['threshold']
        right_indices = ~left_indices
        left_subtree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        return {
            'feature': best_split['feature'],
            'threshold': best_split['threshold'],
            'left': left_subtree,
            'right': right_subtree
        }

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        best_gini = float('inf')
        best_split = None

        for feature_idx in range(n_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_indices = X[:, feature_idx] <= threshold
                right_indices = ~left_indices

                if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                    continue

                gini_left = self._gini_impurity(y[left_indices])
                gini_right = self._gini_impurity(y[right_indices])

                weighted_gini = (np.sum(left_indices) / n_samples) * gini_left + (
                            np.sum(right_indices) / n_samples) * gini_right

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_split = {
                        'feature': feature_idx,
                        'threshold': threshold
                    }
        return best_split

    def _gini_impurity(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, tree):
        if 'value' in tree:
            return tree['value']
        feature_value = x[tree['feature']]
        if feature_value <= tree['threshold']:
            return self._predict_tree(x, tree['left'])
        else:
            return self._predict_tree(x, tree['right'])

    def predict_proba(self, X):
        return np.array([self._predict_proba_tree(x, self.tree) for x in X])

    def _predict_proba_tree(self, x, tree):
        if 'value' in tree:
            return np.bincount([tree['value']], minlength=len(np.unique(y))) / np.sum(
                np.bincount([tree['value']], minlength=len(np.unique(y))))
        feature_value = x[tree['feature']]
        if feature_value <= tree['threshold']:
            return self._predict_proba_tree(x, tree['left'])
        else:
            return self._predict_proba_tree(x, tree['right'])
    
    


# Example usage:
encoder = LabelEncoder()

y_bin = label_binarize(encoder.fit_transform(y),classes=[0,1,2])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)






X_train = np.array(X_train)  # Your training data
y_train = np.array(y_train)  # Your training labels

X_test = np.array(X_test)   # Your test data


nb_classifier = DecisionTreeID3()
nb_classifier.fit(X_train, y_train)


# # Make predictions
predictions = nb_classifier.predict(X_test)


# # Print or use predictions as needed



In [ ]:
predictions[:10]

**METRICS FOR THE MODEL**

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming you have the true labels for the test set
y_true = y_test

# Convert predicted probabilities to predicted labels (using argmax)
# y_pred = y_score.argmax(axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_true,predictions)
print(f'Accuracy: {accuracy:.2f}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true,predictions)
print('Confusion Matrix:')
print(conf_matrix)

# Print classification report
class_names = encoder.classes_
classification_rep = classification_report(y_true,predictions)
print('Classification Report:')
print(classification_rep)


In [ ]:
# probabilities = nb_classifier.predict_proba(X_test)

# # Calculate AUC for each class
# auc_scores = []
# for i, cls in enumerate(nb_classifier.unique_classes):
#     class_labels = (y_test == cls).astype(int)
#     class_probs = probabilities[:, i]
#     auc = roc_auc_score(class_labels, class_probs)
#     auc_scores.append(auc)
#     print(f'AUC for Class {cls}: {auc:.2f}')

# # Plot ROC curves for each class
# plt.figure(figsize=(8, 8))
# for i, cls in enumerate(nb_classifier.unique_classes):
#     class_labels = (y_test == cls).astype(int)
#     class_probs = probabilities[:, i]
#     fpr, tpr, _ = roc_curve(class_labels, class_probs)
#     plt.plot(fpr, tpr, label=f'Class {cls} (AUC = {auc_scores[i]:.2f})')

# plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
# plt.title('Receiver Operating Characteristic (ROC) Curve for Multiclass')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.legend()
# plt.show()

In [ ]:
#Naive Bayesian CLassifier

**TEST DATA
**

In [ ]:
# test_final.head(10)

In [ ]:
# # df = pd.DataFrame(test_final)
# test_final.to_csv('Test1.csv', index=False)

In [ ]:
import joblib
joblib.dump(nb_classifier, 'customm_decision_tree.pkl')
print(f'Download your model [here](sandbox:/kaggle/working/your_model_filename.joblib)')



In [ ]:
import pickle
with open('custom_dt_model.pkl', 'wb') as model_file:
    pickle.dump(nb_classifier, model_file)

In [ ]:
model=joblib.load("dc.pkl")
model.predict(X_test[:10])

In [ ]:
# import csv
# classes=["sh","turn","walk"]
# d_classes_pred = []
# for i in y_pred:
#     class_index = np.argmax(i)
#     d_class = classes[class_index]
#     d_classes_pred.append(d_class)

